# Install doctl


In [1]:
!curl -sL https://github.com/digitalocean/doctl/releases/download/v1.76.0/doctl-1.76.0-linux-amd64.tar.gz | tar -xzv

doctl


In [2]:
!sudo mv doctl /usr/local/bin

# Authenticate to digitalocean using doctl

In [3]:
!doctl auth init –-context function

Please authenticate doctl for use with your DigitalOcean account. You can generate a token in the control panel at https://cloud.digitalocean.com/account/api/tokens

Enter your access token: 
Validating token... OK



# Install serverless in doctl, so we can make functions

In [4]:
!doctl serverless install

Downloading...Unpacking...Installing...Cleaning up...
Done


Create a directory to work in

In [5]:
!unzip faas.zip

Archive:  faas.zip
   creating: faas/
   creating: faas/.do/
  inflating: faas/.do/deploy.template.yaml  
   creating: faas/packages/
   creating: faas/packages/speech-to-text/
   creating: faas/packages/speech-to-text/speech-to-text/
  inflating: faas/packages/speech-to-text/speech-to-text/build.sh  
  inflating: faas/packages/speech-to-text/speech-to-text/requirements.txt  
  inflating: faas/packages/speech-to-text/speech-to-text/__main__.py  
  inflating: faas/project.yml        
  inflating: faas/sort.py            


In [7]:
%cd faas

/content/faas


In [8]:
!pwd

/content/faas


# Connect to your digitalocean sandbox

In [9]:

!doctl sandbox connect

Connected to function namespace 'fn-6398607d-1b12-4842-8668-ad0c975abdbe' on API host 'https://faas-blr1-8177d592.doserverless.co'



# Build and test your digitalocean function

Build

In [305]:
!chmod +x packages/speech-to-text/speech-to-text/build.sh

In [306]:
!doctl sandbox deploy . --remote-build

Deploying '/content/faas'
  to namespace 'fn-6398607d-1b12-4842-8668-ad0c975abdbe'
  on host 'https://faas-blr1-8177d592.doserverless.co'
Submitted action 'speech-to-text' for remote building and deployment in runtime python:default (id: 62f76c2bfb7f4a8bb76c2bfb7fda8bbd)
Processing of 'speech-to-text' is still running remotely ...
Processing of 'speech-to-text' is still running remotely ...
Processing of 'speech-to-text' is still running remotely ...
Processing of 'speech-to-text' is still running remotely ...

Deployed functions ('doctl sbx fn get <funcName> --url' for URL):
  - speech-to-text/speech-to-text


In [307]:
!doctl serverless status

Connected to function namespace 'fn-6398607d-1b12-4842-8668-ad0c975abdbe' on API host 'https://faas-blr1-8177d592.doserverless.co'
Sandbox version is 3.1.1-1.2.1



In [308]:
import requests, json
import base64
import io
url = !doctl sbx fn get speech-to-text/speech-to-text --url
print(url[0])
with io.open('/content/speech2.wav', "rb") as audio_file:
        file=str(base64.b64encode(audio_file.read())).strip("b/").replace("'", '')
y = requests.post(url[0],data={"lang":"en-US","audio":file})


https://faas-blr1-8177d592.doserverless.co/api/v1/web/fn-6398607d-1b12-4842-8668-ad0c975abdbe/speech-to-text/speech-to-text


In [309]:
y.text

'results {\n  alternatives {\n    transcript: "Buy CDs at 12, dramatic."\n    confidence: 0.802226067\n  }\n  result_end_time {\n    seconds: 1\n    nanos: 590000000\n  }\n  language_code: "en-us"\n}\ntotal_billed_time {\n  seconds: 2\n}\n'

In [314]:
!zip -r fass_updated.zip faas

  adding: faas/ (stored 0%)
  adding: faas/packages/ (stored 0%)
  adding: faas/packages/speech-to-text/ (stored 0%)
  adding: faas/packages/speech-to-text/speech-to-text/ (stored 0%)
  adding: faas/packages/speech-to-text/speech-to-text/__main__.py (deflated 44%)
  adding: faas/packages/speech-to-text/speech-to-text/requirements.txt (stored 0%)
  adding: faas/packages/speech-to-text/speech-to-text/build.sh (deflated 34%)
  adding: faas/packages/speech-to-text/speech-to-text/paramount-371207-4576b089b156.json (deflated 30%)
  adding: faas/.do/ (stored 0%)
  adding: faas/.do/deploy.template.yaml (deflated 30%)
  adding: faas/.ipynb_checkpoints/ (stored 0%)
  adding: faas/sort.py (deflated 51%)
  adding: faas/project.yml (deflated 32%)
  adding: faas/.nimbella/ (stored 0%)
  adding: faas/.nimbella/versions.json (deflated 36%)


In [ ]:
with open('json_data.json', 'w') as outfile:
    json.dump(x, outfile)
!cat json_data.json

{"0": "0.9430294816720703", "1": "0.9430294816720703", "2": "0.15083726450822088", "3": "0.15083726450822088"}

We can also test directly from the command line

In [ ]:
!doctl sls fn invoke sample/hello -P json_data.json

{
  "body": "2.603903891418075"
}


In [ ]:
!doctl sandbox activations logs

=== af23e79994314edaa3e7999431ceda66 (success) 05/25 13:49:33 hello:0.0.10
